In [11]:
import os, requests, torch, transformers, warnings
from PIL import Image
from torchvision import transforms
from io import BytesIO


ckpt_name = 'aehrc/mimic-cxr-report-gen-single'

cache_dir = '/scratch/pawsey0864/anicolson/checkpoints'

os.environ['TRANSFORMERS_CACHE'] = cache_dir
os.environ['HUGGINGFACE_HUB_CACHE'] = cache_dir

encoder_decoder = transformers.AutoModel.from_pretrained(ckpt_name, trust_remote_code=True, cache_dir=cache_dir)
tokenizer = transformers.PreTrainedTokenizerFast.from_pretrained(ckpt_name, cache_dir=cache_dir)
image_processor = transformers.AutoFeatureExtractor.from_pretrained(ckpt_name, cache_dir=cache_dir)

test_transforms = transforms.Compose(
    [
        transforms.Resize(size=image_processor.size['shortest_edge']),
        transforms.CenterCrop(size=[
            image_processor.size['shortest_edge'],
            image_processor.size['shortest_edge'],
        ]
        ),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=image_processor.image_mean,
            std=image_processor.image_std,
        ),
    ]
)

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


In [10]:
url = 'https://www.stritch.luc.edu/lumen/meded/radio/curriculum/IPM/PCM/86a_labelled.jpg'
response = requests.get(url)
image_a = Image.open(BytesIO(response.content))
image_a = image_a.convert('RGB')

url = 'https://prod-images-static.radiopaedia.org/images/566180/d527ff6fc1482161c9225345c4ab42_big_gallery.jpg'
response = requests.get(url)
image_b = Image.open(BytesIO(response.content))
image_b = image_b.convert('RGB')

image_a = test_transforms(image_a)
image_b = test_transforms(image_b)

images = torch.stack([image_a, image_b], dim=0)
images.shape


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:17                                                                                   │
│                                                                                                  │
│   14 image_a = torch.randn(3, 500, 600)                                                          │
│   15 image_b = torch.randn(3, 1234, 1111)                                                        │
│   16                                                                                             │
│ ❱ 17 image_a = test_transforms(image_a)                                                          │
│   18 image_b = test_transforms(image_b)                                                          │
│   19                                                                                             │
│   20 images = torch.stack([image_a, image_b], dim=0)                                             │
│                                                                                                  │
│ /software/projects/pawsey0864/anicolson/environments/multimodal_23/lib/python3.10/site-packages/ │
│ torchvision/transforms/transforms.py:95 in __call__                                              │
│                                                                                                  │
│     92 │                                                                                         │
│     93 │   def __call__(self, img):                                                              │
│     94 │   │   for t in self.transforms:                                                         │
│ ❱   95 │   │   │   img = t(img)                                                                  │
│     96 │   │   return img                                                                        │
│     97 │                                                                                         │
│     98 │   def __repr__(self) -> str:                                                            │
│                                                                                                  │
│ /software/projects/pawsey0864/anicolson/environments/multimodal_23/lib/python3.10/site-packages/ │
│ torchvision/transforms/transforms.py:137 in __call__                                             │
│                                                                                                  │
│    134 │   │   Returns:                                                                          │
│    135 │   │   │   Tensor: Converted image.                                                      │
│    136 │   │   """                                                                               │
│ ❱  137 │   │   return F.to_tensor(pic)                                                           │
│    138 │                                                                                         │
│    139 │   def __repr__(self) -> str:                                                            │
│    140 │   │   return f"{self.__class__.__name__}()"                                             │
│                                                                                                  │
│ /software/projects/pawsey0864/anicolson/environments/multimodal_23/lib/python3.10/site-packages/ │
│ torchvision/transforms/functional.py:140 in to_tensor                                            │
│                                                                                                  │
│    137 │   if not torch.jit.is_scripting() and not torch.jit.is_tracing():                       │
│    138 │   │   _log_api_usage_once(to_tensor)                                                    │
│    139 │   if not (F_pil._is_pil_image(pic) or _is_numpy(pic)):                                  │
│ ❱  140 │   │   raise TypeError(f"pic should be PIL Image or

In [9]:
outputs = encoder_decoder.generate(
    pixel_values=images,
    special_token_ids=[tokenizer.sep_token_id],
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id,
    return_dict_in_generate=True,
    use_cache=True,
    max_length=256,
    num_beams=4,
)

findings, impression = encoder_decoder.split_and_decode_sections(
    outputs.sequences,
    [tokenizer.sep_token_id, tokenizer.eos_token_id],
    tokenizer,
)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│    1 outputs = encoder_decoder.generate(                                                         │
│ ❱  2 │   pixel_values=images,                                                                    │
│    3 │   special_token_ids=[tokenizer.sep_token_id],                                             │
│    4 │   bos_token_id=tokenizer.bos_token_id,                                                    │
│    5 │   eos_token_id=tokenizer.eos_token_id,                                                    │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'images' is not defined

In [4]:
for i, j in zip(findings, impression):
    print(f'Findings: {i}\nImpression: {j}\n')

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 for i, j in zip(findings, impression):                                                       │
│   2 │   print(f'Findings: {i}\nImpression: {j}\n')                                               │
│   3                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'findings' is not defined